<a href="https://colab.research.google.com/github/basturkme/4IER-HMI/blob/PokeD/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import scipy.io
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# --- 1. AYARLAR ---
dosya_yolu = '/S1_A1_E1.mat'
MODEL_ADI = 'robust_emg_model'

# Veriyi Yükle
if not os.path.exists(dosya_yolu):
    raise SystemExit("Dosya bulunamadı!")

mat = scipy.io.loadmat(dosya_yolu)
df = pd.DataFrame(mat['emg'])
df['Label'] = mat['restimulus']

# Hedef: İşaret Parmağı (1)
df_hareket = df[df['Label'] == 1].copy()
df_dinlenme = df[df['Label'] == 0].copy()

# Eşitleme (Undersampling)
if len(df_hareket) > 0:
    df_dinlenme_esit = df_dinlenme.sample(n=len(df_hareket), random_state=42)
    df_filtered = pd.concat([df_hareket, df_dinlenme_esit])
    df_filtered['Label'] = df_filtered['Label'].apply(lambda x: 1 if x == 1 else 0)
    df_filtered = df_filtered.sample(frac=1, random_state=42).reset_index(drop=True)
else:
    raise SystemExit("Hedef hareket bulunamadı")

X = df_filtered[[0, 1, 2, 3]].values
y = df_filtered['Label'].values

# Gürültü Ekleme (Sağlamlık için - overfitting'i engeller)
noise = np.random.normal(0, 0.05, X.shape)
X_noisy = X + noise

# Normalizasyon
scaler = StandardScaler()
X_train = scaler.fit_transform(X_noisy) # Gürültülü veriyi eğit
# Test için temiz veriyi kullanacağız

# --- 2. GÜÇLÜ DENSE MODEL (CNN Yerine Bunu Kullanın) ---
# Bu mimari ESP32'de ASLA nan vermez.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1), # Hafif unutma (Ezber bozar)
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y, epochs=30, batch_size=16, verbose=1)

# --- 3. DOSYALARI OLUŞTUR ---
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

def hex_to_c_array(data, var_name):
    c_str = ''
    for i, val in enumerate(data):
        c_str += f'0x{val:02x}, '
        if (i + 1) % 12 == 0: c_str += '\n  '
    return f'const unsigned char {var_name}[] = {{\n  {c_str[:-2]}\n}};\nconst unsigned int {var_name}_len = {len(data)};'

with open(f'{MODEL_ADI}_data.h', 'w') as f:
    f.write(hex_to_c_array(tflite_model, 'emg_model_data'))

# Test Vektörü (Garanti Senaryo)
X_clean = df[[0, 1, 2, 3]].values
X_clean_norm = scaler.transform(X_clean)

# Dinlenme ve Hareket indeksleri
idx_rest = np.where(df['Label'] == 0)[0]
idx_move = np.where(df['Label'] == 1)[0]

# Sıralı Senaryo: Dinlenme -> Hareket -> Dinlenme -> Hareket
# (Indexlerin taşmadığından emin olmak için güvenli aralıklar)
test_seq = np.concatenate((
    X_clean_norm[idx_rest[0:50]],
    X_clean_norm[idx_move[100:150]],
    X_clean_norm[idx_rest[200:250]],
    X_clean_norm[idx_move[300:350]]
))

def generate_test_header(data):
    c_str = f'const int TEST_DATA_LEN = {len(data)};\nconst float test_data[][4] = {{\n'
    for row in data:
        c_str += '    {' + ', '.join([f'{x:.4f}' for x in row]) + '},\n'
    c_str += '};\n'
    return c_str

with open('test_vectors.h', 'w') as f:
    f.write("#ifndef TEST_VECTORS_H\n#define TEST_VECTORS_H\n\n")
    f.write(generate_test_header(test_seq))
    f.write("\n#endif")

from google.colab import files
files.download(f'{MODEL_ADI}_data.h')
files.download('test_vectors.h')

Epoch 1/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6934 - loss: 0.5575
Epoch 2/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7857 - loss: 0.4420
Epoch 3/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7928 - loss: 0.4391
Epoch 4/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7963 - loss: 0.4337
Epoch 5/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7902 - loss: 0.4374
Epoch 6/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7946 - loss: 0.4347
Epoch 7/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7953 - loss: 0.4321
Epoch 8/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7857 - loss: 0.4474
Epoch 9/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7906 - loss: 0.4396
Epoch 10/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7955 - loss: 0.4273
Epoch 11/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7919 - loss: 0.4378
Epoch 12/30
477/477 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>